In [1111]:
import pandas as pd
import numpy as np

cars = pd.read_csv("./concatenated_data.csv", index_col=0)
del cars["description"]


In [1112]:
# prix -> abs(prix)

cars["prix"] = abs(cars["prix"])


In [1113]:
# kilometrage -> abs(kilometrage)

cars["kilometrage"] = abs(cars["kilometrage"])


In [1114]:
# if less than 500 -> *= 1000

less_than_1000 = cars["prix"] <= 500
cars.loc[less_than_1000, "prix"] *= 1000


In [1115]:
# if over 1 000 000 -> turn prix to nan
over_1000000 = cars["prix"] >= 1000000
cars.loc[over_1000000, "prix"] = np.nan


In [1116]:
# if prix is 0 -> turn prix to nan
prix_is_0 = cars["prix"] == 0
cars.loc[prix_is_0, "prix"] = np.nan


In [1117]:
# if kilometrage >= 1 000 000 -> turn to nan

kilometrage_more_than_1000000 = cars["kilometrage"] >= 1000000
cars.loc[kilometrage_more_than_1000000, "kilometrage"] = np.nan


In [1118]:
# applying str.title() to each column

for col in cars.columns:
    if cars[col].dtype == "object":
        cars[col] = cars[col].str.title()


In [1119]:
# exploring the values of cars["localisation"]

localisation_values = cars["localisation"].unique()
print(localisation_values, len(localisation_values))


['Ariana' 'Tunis' 'Ben Arous' 'Bizerte' 'Monastir' 'Béja' 'La Manouba'
 'Sousse' 'Nabeul' 'Sidi Bouzid' 'Gafsa' 'Kairouan' 'Sfax' 'Mahdia'
 'Tozeur' 'Médenine' 'Kasserine' 'Le Kef' 'Gabès' 'Zaghouan' 'Jendouba'
 'Kébili' 'Siliana' 'Manouba' 'Gabes' 'Tataouine' nan] 27


In [1120]:
# fix locations' names

my_dict = {
    "Béja": "Beja",
    "La Manouba": "Manouba",
    "Gabès": "Gabes",
    "Médenine": "Medenine",
    "Kébili": "Kebili",
}
cars.replace({"localisation": my_dict}, inplace=True)


In [1121]:
# remove month from year in the column annee and convert it to float64

contains_hyphen = cars["annee"].str.contains("-") & cars["annee"].notna()
cars.loc[contains_hyphen, "annee"] = cars.loc[contains_hyphen, "annee"].apply(
    lambda string: string.split("-")[1]
)
contains_point = cars["annee"].str.contains("[.]") & cars["annee"].notna()
cars.loc[contains_point, "annee"] = cars.loc[contains_point, "annee"].apply(
    lambda string: string.split(".")[-1]
)
cars["annee"] = cars["annee"].astype(np.float64)


In [1122]:
# if 10<= annee <= 23 -> annee + 2000, if 60 <= annee -> annee + 1900, else turn to nan

annee_between_0_and_23 = (cars["annee"] >= 10) & (cars["annee"] <= 23)
cars.loc[annee_between_0_and_23, "annee"] += 2000
annee_more_than_60 = (cars["annee"] >= 60) & (cars["annee"] <= 99)
cars.loc[annee_more_than_60, "annee"] += 1900
correct_annee = (cars["annee"] >= 1960) & (cars["annee"] <= 2023)
cars.loc[~correct_annee, "annee"] = np.nan


In [1123]:
# convert N.D to nan in the "puissance" column

is_ND = cars["puissance"] == "N.D"
cars.loc[is_ND, "puissance"] = np.nan


In [1124]:
# convert the column puissance to float

contains_CV = cars["puissance"].str.contains("C") & cars["puissance"].notna()
cars.loc[contains_CV, "puissance"] = cars.loc[contains_CV, "puissance"].apply(
    lambda string: string.split()[0]
)
cars["puissance"] = cars["puissance"].astype(np.float64)


In [1125]:
# if puissance > 200 -> turn to nan

puissance_more_than_200 = cars["puissance"] > 200
cars.loc[puissance_more_than_200, "puissance"] = np.nan


In [1126]:
# convert the column cylindree to float

contains_symbol = cars["cylindree"].str.contains("[><]") & cars["cylindree"].notna()
cars.loc[contains_symbol, "cylindree"] = cars.loc[contains_symbol, "cylindree"].apply(
    lambda string: string[1:]
)
cars.loc[cars["cylindree"].notna(), "cylindree"] = cars.loc[
    cars["cylindree"].notna(), "cylindree"
].apply(lambda string: string[:-1])
cars["cylindree"] = cars["cylindree"].astype(np.float64)


In [1127]:
cars.to_csv("new_clean_data.csv")
